In [8]:
!pip install datasets
!pip install Pillow
!pip install numpy
!pip install matplotlib

In [10]:
!git clone https://github.com/clovaai/CRAFT-pytorch.git

fatal: destination path 'CRAFT-pytorch' already exists and is not an empty directory.


In [2]:
!pip install opencv-python==4.5.4.60
!pip install torch
!pip install torchvision
!pip install scikit-image
!pip install scipy
!pip install craft-text-detector
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 11.4 MB/s eta 0:00:00
  Using cached craft_text_detector-0.4.3-py3-none-any.whl.metadata (4.8 kB)
  Using cached opencv-python-4.5.4.60.tar.gz (89.8 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
from datasets import load_dataset
import easyocr
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from craft_text_detector import Craft

# Path ke file pre-trained model di Google Drive
pretrained_model_path = '/content/drive/My Drive/skripsi/craft_mlt_25k.pth'

# Inisialisasi Craft dengan model pre-trained
craft = Craft(
    output_dir='outputs/',
    crop_type="poly",
    cuda=False,  # Gunakan cuda=True jika pakai GPU
    model_path=pretrained_model_path  # Tentukan path ke model pre-trained
)

ModuleNotFoundError: No module named 'craft_text_detector'

**Text Extraction**

> using easyocr

> support bahasa indonesia and english





In [ ]:
def detect_text_craft(image):
    # Jika gambar berupa PIL Image, konversi ke NumPy array
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Jika gambar masih dalam mode grayscale atau alpha, ubah ke BGR
    if len(image.shape) == 2 or image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Jalankan deteksi teks
    prediction_result = craft.detect_text(image)

    # Ambil bounding boxes
    boxes = prediction_result["boxes"]

    # Gambar bounding box
    for box in boxes:
        pts = box.reshape((-1, 1, 2)).astype(int)
        cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Tampilkan hasil
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

    return boxes

In [ ]:
reader = easyocr.Reader(['en', 'id'])

def is_receipt(text, logic='AND'):
    required_keywords = ['total', 'amount', 'jumlah', 'price', 'harga']
    text_lower = text.lower()

    #diubah nanti jadinya ada yang and ada yang or
    if logic == 'AND':
        return all(keyword in text_lower for keyword in required_keywords)
    elif logic == 'OR':
        return any(keyword in text_lower for keyword in required_keywords)
    else:
        raise ValueError("Invalid logic parameter. Use 'AND' or 'OR'.")


def process_image(image, logic='AND'):
    # Convert PIL Image to numpy array
    image_np = np.array(image)

    # Extract text using EasyOCR
    results = reader.readtext(image_np)

    # Combine all detected text into a single string
    extracted_text = ' '.join([result[1] for result in results])

    # Check if the extracted text is a receipt
    if is_receipt(extracted_text, logic=logic):
        print("This is a valid receipt.")
        print("Extracted Text:")
        print(extracted_text)
    else:
        print("This is not a valid receipt.")

In [ ]:
# Load the CORD-v2 dataset
dataset = load_dataset("naver-clova-ix/cord-v2")

# Example: Process the first few images in the dataset
for example in dataset['train'].select(range(5)):  # Process the first 5 examples
    image = example['image']  # Get the PIL Image object
    print("Processing image...")

    # Process the image with OR logic
    detect_text_east(image)
    process_image(image, logic='OR')
    print("-" * 50)

# Load international style dataset
dataset_international = load_dataset("katanaml-org/invoices-donut-data-v1")

# Example: Process the first few images in the dataset_international
for example_int in dataset_international['train'].select(range(5)):  # Process the first 5 examples
    image_int = example_int['image']
    print("Processing image international...")

    # Process the image with OR logic
    detect_text_east(image_int)
    process_image(image_int, logic='OR')
    print("-" * 50)